# <center><font color=black> NFL Data Science </font></center>
## <center><font color= black> Using FootballDB.com Data </font></center>
### <center> David Armah Jr. </center>

### WebScrape / Data Cleaning Process

In [1]:
#Necessary Imports for this Section
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib

In [2]:
## Configure and Collect Data
## Retrieve Data from given url
# Return data in a pandas DataFrame
def configure(url):
  
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    data = requests.get(url, headers=hdr)
    soup = BeautifulSoup(data.text, 'html.parser')
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    return df

## Cleans the Players Data, fixing all unnessecary data 
def clean_playerData(df):
    #df['Player'] = df['Player'].str.split(".").str[0]
    #df['Player'] = df['Player'].str[:-1]
    #df.index = df['Player']
    for i in df['Player']:
        player = i
        #print(player)
        
        L = False
        R = 0
        
        while R < len(player):
            
            if player[R] == ' ' and L == False:
                L = True
            elif player[R] == '.' and L == True:
                df['Player'] = df['Player'].replace(player, player[:R])
                #df['Player'] = df['Player'].str[:-1]
                break
            R += 1
    df['Player'] = df['Player'].str[:-1]
    df.index = df['Player']

#Cleans the Team Data, fixing all unnecessary data
def clean_teamData(df):
    for i in df['Team']:
        team = i
    
        L = False
        R =0
    
        while R < len(team):
            if (team[R].isupper() or team[R].isdigit()) and L == False:
                L = True    
            elif team[R].isupper() and L == True:
                df['Team']= df['Team'].replace(team, team[:R])
                break    
            elif team[R] == " ":
                L = False
            R += 1  
    df.index = df['Team']

In [5]:
def player_test(year):
    for i in year:
        #passing
        passing_stats = 'https://www.footballdb.com/statistics/nfl/player-stats/passing/' + str(year) + '/regular-season?sort=passrate&limit=all'
        #rushing
        rushing_stats = 'https://www.footballdb.com/statistics/nfl/player-stats/rushing/' + str(year) + '/regular-season?sort=rushyds&limit=all'
        #receiving
        receiving_stats = 'https://www.footballdb.com/statistics/nfl/player-stats/receiving/' + str(year) + '/regular-season?sort=recnum&limit=all'
        player_test = [passing_stats,rushing_stats, receiving_stats]
        player_list = []
        for i in player_test:
            curr_df = configure(i)
            clean_playerData(curr_df)
            player_list.append(curr_df)
        return player_list
def team_test(year):
    for i in year: 
        #team offense
        offensive_stats = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=' + str(year) + '&type=reg&cat=T&group=O&conf='
        #team defense
        defensive_stats = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=' + str(year) + '&type=reg&cat=T&group=D&conf='
        #play selection data
        plays_stats = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=' + str(year) + '&type=reg&cat=S&group=O&conf='
        team_test = [offensive_stats,defensive_stats,plays_stats ]
        team_list = []
        for i in team_test:
            curr_df = configure(i)
            clean_teamData(curr_df)
            team_list.append(curr_df)
        return team_list
    

In [ ]:
players_2022 = player_test([2022])
teams_2022 = team_test([2022])

In [ ]:
passing_2022 = players_2022[0]
rushing_2022 = players_2022[1]
receiving_2022 = players_2022[2]
teamOffense_2022 = teams_2022[0]
teamDefense_2022 = teams_2022[1]
teamPlays_2022 = teams_2022[2]

In [ ]:
# create data for average statistics

avg_passing_2022 = passing_2022.copy()
avg_rushing_2022 = rushing_2022.copy()
avg_receiving_2022 = receiving_2022.copy()

In [ ]:
def get_averages(passing, rushing, receiving):
    passing = passing.drop(columns=['YPA', 'TD%T%','Int%I%','Lg','Sack', 'Loss', 'Rate'])
    rushing = rushing.drop(columns=['Avg', 'YPG','Lg' ])
    receiving = receiving.drop(columns=['Avg', 'YPG', 'Lg'])
    passing[['Att', 
                  'Cmp', 'Pct', 'Yds', 'TD', 'Int']] = passing[[ 'Att', 
                                                                         'Cmp', 'Pct', 'Yds', 'TD', 'Int']].div(passing['Gms'].values,axis=0)
    rushing[[ 'Att', 'Yds', 
                  'TD', 'FD']] = rushing[['Att', 'Yds', 
                                                   'TD', 'FD']].div(rushing['Gms'].values,axis=0)
    receiving[[ 'Rec', 'Yds', 
                    'TD', 'FD', 'Tar', 'YAC']] = receiving[[ 'Rec', 'Yds', 
                                                                     'TD', 'FD', 'Tar', 'YAC']].div(receiving['Gms'].values,axis=0)
    return passing, rushing, receiving



    

In [ ]:
pass_avg, rush_avg, rec_avg = get_averages(avg_passing_2022, avg_rushing_2022, avg_receiving_2022 )

In [ ]:
rec_avg